# Summary of sensitivity results test

In [1]:
%load_ext autoreload
%autoreload 2
%cd D:\synthetic-sweden
%matplotlib inline

D:\synthetic-sweden


In [2]:
import pandas as pd
from tqdm import tqdm
pd.options.display.float_format = '{:,.2f}'.format
scenario = 'scenario_vg_car'

## 1. Individual statistics

In [60]:
df_indi = pd.read_csv(f'results/sensitivity/{scenario}_stats_indi_agg_5days.csv')
tqdm.pandas()
df_indi.loc[:, 'Charging_type'] = df_indi.progress_apply(lambda row: str(int(row['charging_type'])) + ' (' + str(row['type1']) + ')' if row['charging_type'] in (1, 2) else str(int(row['charging_type'])) + ' (0.9)', axis=1)
df_indi.head()

100%|██████████| 20/20 [00:00<?, ?it/s]


,paraset,charging_type,home_charger,failure_rate,charging_share,num_charged,soc_end,charging_time_inter,charging_time_fast,charging_time_ratio,energy_fast,energy_inter,energy_total,type1,type2,type3,power_fast,min_parking_time,power_intermediate,Charging_type
0,0,1,0,0.94,3.60,"10,436.00",0.50,103.79,1.16,0.29,2.31,46.98,"514,437.29",0.20,0.20,0.90,150,5,22,1 (0.2)
1,0,1,1,1.17,0.35,"1,001.00",0.88,102.13,0.70,0.36,1.40,47.01,"48,456.87",0.20,0.20,0.90,150,5,22,1 (0.2)
2,0,2,0,0.67,7.29,"21,133.00",0.53,106.63,0.76,0.31,1.51,45.37,"990,711.48",0.20,0.20,0.90,150,5,22,2 (0.2)
3,0,2,1,0.37,1.91,"5,532.00",0.88,90.87,0.34,0.31,0.68,33.50,"189,096.67",0.20,0.20,0.90,150,5,22,2 (0.2)
4,0,3,0,0.57,18.23,"52,868.00",0.91,50.60,0.18,0.12,0.36,15.62,"845,013.55",0.20,0.20,0.90,150,5,22,3 (0.9)


### 1.1 Share of failed agents

In [61]:
table = pd.pivot_table(df_indi, values='failure_rate', index=['Charging_type', 'power_fast'],
                    columns=['home_charger'])
table.to_clipboard()
table

home_charger                0    1
Charging_type power_fast          
1 (0.2)       50         0.96 1.17
              150        0.94 1.17
1 (0.3)       50         0.80 0.99
              150        0.79 0.98
2 (0.2)       50         0.68 0.37
              150        0.67 0.37
2 (0.3)       50         0.60 0.37
              150        0.60 0.37
3 (0.9)       50         0.57 0.37
              150        0.57 0.37

### 1.2 Share of agents who accessed daytime charging

In [62]:
table = pd.pivot_table(df_indi, values='charging_share', index=['Charging_type', 'power_fast'],
                    columns=['home_charger'])
table.to_clipboard()
table

home_charger                 0     1
Charging_type power_fast            
1 (0.2)       50          3.59  0.35
              150         3.60  0.35
1 (0.3)       50          6.70  0.63
              150         6.66  0.63
2 (0.2)       50          7.36  1.91
              150         7.29  1.91
2 (0.3)       50          8.17  2.53
              150         8.05  2.53
3 (0.9)       50         18.21 22.44
              150        18.23 22.44

### 1.3 Charging time and ratio, energy / person and total energy

In [44]:
df_indi_energy = pd.read_csv(f'results/sensitivity/{scenario}_stats_indi_5days.csv')
df_indi_energy.head()

,person,car,home_charger,residential_charger,soc_end,Charging_type,power_intermediate,power_fast,charging_time_fast,charging_time_inter,charging_time_ratio,charging_energy_fast,charging_energy_inter,charging_energy_overnight
0,10000013,B,1,0,0.78,1 (0.2),22,50,0.00,0.00,0.00,0.00,0.00,8.72
1,10000013,B,1,0,0.78,2 (0.2),22,50,0.00,0.00,0.00,0.00,0.00,8.72
2,10000013,B,1,0,0.89,3 (0.9),22,50,0.00,15.53,0.03,0.00,4.27,4.45
3,10000025,C,1,0,0.77,1 (0.2),22,50,0.00,0.00,0.00,0.00,0.00,13.62
4,10000025,C,1,0,0.77,2 (0.2),22,50,0.00,0.00,0.00,0.00,0.00,13.62


In [49]:
def data_summary(data):
    data2compute_inter = data.loc[data['charging_time_inter']>0, :]
    data2compute_fast = data.loc[data['charging_time_fast']>0, :]
    sum_dict = dict(
        charging_time_inter = data2compute_inter['charging_time_inter'].median(),
        charging_time_ratio_inter = data2compute_inter['charging_time_ratio'].median()*100,
        charging_time_fast = data2compute_fast['charging_time_fast'].median(),
        charging_time_ratio_fast = data2compute_fast['charging_time_ratio'].median()*100,
        energy_daytime=(data['charging_energy_fast'].sum() + data['charging_energy_inter'].sum())/10**3,
        energy_overnight=data['charging_energy_overnight'].sum()/10**3
    )
    return pd.Series(sum_dict)

In [65]:
df_indi_e = df_indi_energy.groupby(['Charging_type', 'power_fast', 'home_charger']).apply(data_summary).reset_index() # .to_clipboard()
table = pd.pivot_table(df_indi_e, values=['charging_time_inter', 'charging_time_ratio_inter', 'charging_time_fast', 'charging_time_ratio_fast',
                                          'energy_daytime', 'energy_overnight'], index=['Charging_type', 'power_fast'],
                    columns=['home_charger'])
table.to_clipboard()
table

charging_time_fast       charging_time_inter         \
home_charger                              0     1                   0      1   
Charging_type power_fast                                                       
1 (0.2)       50                      18.97 17.50              115.80 104.85   
              150                     15.60 14.45              115.81 104.85   
1 (0.3)       50                      18.87 20.09              108.07 101.34   
              150                     15.28 15.22              108.15 101.33   
2 (0.2)       50                      17.85 12.47              108.73  85.15   
              150                     13.02  7.66              108.85  85.09   
2 (0.3)       50                      16.89 11.13              102.21  79.90   
              150                     12.40  6.66              101.93  79.85   
3 (0.9)       50                       5.12  5.27               41.30  42.32   
              150                      1.74  1.89               41.31  42.32   

                         charging_time_ratio_fast        \
home_charger                                    0     1   
Charging_type power_fast                                  
1 (0.2)       50                             7.20 11.48   
              150                            6.27  7.29   
1 (0.3)       50                             9.92 10.05   
              150                            8.09  7.66   
2 (0.2)       50                            21.07 34.20   
              150                           15.77 29.36   
2 (0.3)       50                            22.37 34.21   
              150                           15.89 27.76   
3 (0.9)       50                            23.69 14.09   
              150                            9.98 11.29   

                         charging_time_ratio_inter       energy_daytime  \
home_charger                                     0     1              0   
Charging_type power_fast                                                  
1 (0.2)       50                             30.08 37.04         511.08   
              150                            30.18 37.04         514.44   
1 (0.3)       50                             29.91 33.31         901.37   
              150                            29.96 33.30         905.69   
2 (0.2)       50                             30.90 31.45         996.34   
              150                            30.91 31.29         990.71   
2 (0.3)       50                             29.39 28.72         999.31   
              150                            29.45 28.60         991.36   
3 (0.9)       50                             12.43 12.16         842.74   
              150                            12.43 12.15         845.01   

                                energy_overnight           
home_charger                  1                0        1  
Charging_type power_fast                                   
1 (0.2)       50          48.24           457.78 2,511.87  
              150         48.46           459.37 2,511.66  
1 (0.3)       50          82.20           335.36 2,477.91  
              150         82.80           331.58 2,477.32  
2 (0.2)       50         188.60           164.97 2,371.52  
              150        189.10           162.97 2,371.02  
2 (0.3)       50         230.79           160.32 2,329.33  
              150        231.31           158.77 2,328.81  
3 (0.9)       50         985.24           158.39 1,574.87  
              150        986.09           155.87 1,574.02

## 2. Spatiotemporal statistics

### 2.1 Charger in DeSO zones and aggregate statistics

In [66]:
df_charger = pd.read_csv(f'results/sensitivity/{scenario}_stats_charger_5days.csv')
df_charger.head()

,deso,Charging_type,Purpose,power_fast,charger,number
0,0114C1050,1 (0.2),Other,50,22,1.00
1,0114C1050,1 (0.2),Other,150,22,1.00
2,0114C1050,1 (0.3),Other,50,22,1.00
3,0114C1050,1 (0.3),Other,150,22,1.00
4,0114C1050,2 (0.2),Other,50,22,1.00


In [67]:
df_agg = df_charger.groupby(['Charging_type', 'Purpose', 'power_fast', 'charger'])['number'].sum().reset_index().drop_duplicates(subset=['Charging_type', 'Purpose', 'power_fast', 'charger'])
table = pd.pivot_table(df_agg, values='number', index=['Charging_type','power_fast', 'Purpose'], columns=['charger'])
table.to_clipboard()
table

charger                                22       50       150
Charging_type power_fast Purpose                            
1 (0.2)       50         Other    3,471.00   509.00      NaN
                         Work     1,895.00     8.00      NaN
              150        Other    3,457.00      NaN   540.00
                         Work     1,855.00      NaN     8.00
1 (0.3)       50         Other    5,352.00   754.00      NaN
                         Work     3,762.00    19.00      NaN
              150        Other    5,335.00      NaN   739.00
                         Work     3,757.00      NaN    14.00
2 (0.2)       50         Other    8,092.00   787.00      NaN
                         Work     4,596.00    30.00      NaN
              150        Other    8,065.00      NaN   800.00
                         Work     4,516.00      NaN    32.00
2 (0.3)       50         Other    8,807.00   899.00      NaN
                         Work     4,955.00    43.00      NaN
              150        Other    8,825.00      NaN   842.00
                         Work     4,842.00      NaN    37.00
3 (0.9)       50         Other   14,542.00 1,250.00      NaN
                         Work     9,475.00    70.00      NaN
              150        Other   14,526.00      NaN 1,121.00
                         Work     9,465.00      NaN    68.00

Overnight chargers

In [68]:
df_indi_raw = pd.read_csv(f'results/sensitivity/{scenario}_indi_raw_5days.csv')
df_indi_raw.head()

,person,charging_type,home_charger,finish_day,soc_init,soc_end,total_distance,parking_dur,paraset,residential_charger
0,10000013,1,1,1,1.00,0.78,54.08,617.45,1,0
1,10000013,2,1,1,1.00,0.78,54.08,617.45,1,0
2,10000013,3,1,1,1.00,0.89,54.08,617.45,1,0
3,10000025,1,1,1,1.00,0.77,71.93,600.02,1,0
4,10000025,2,1,1,1.00,0.77,71.93,600.02,1,0


In [69]:
df_indi_raw.groupby(['charging_type', 'paraset'])['home_charger', 'residential_charger'].sum()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


home_charger  residential_charger
charging_type paraset                                   
1             0              175715                11348
              1              175715                11290
              2              175715                 4983
              3              175715                 5051
2             0              175715                 2060
              1              175715                 2101
              2              175715                 1969
              3              175715                 2001
3             0              175715                 1905
              1              175715                 1962

### 2.2 Temporal power demand
Visualisation to present the results.

In [38]:
df_power = pd.read_csv(f'results/sensitivity/{scenario}_stats_power_5days.csv')
df_power.head(70)

,minute,Charging_type,Purpose,power_fast,charger,cars,power
0,0.00,1 (0.2),Other,50,22,0.00,0.00
1,0.00,1 (0.2),Other,50,50,0.00,0.00
2,0.00,1 (0.2),Other,150,22,0.00,0.00
3,0.00,1 (0.2),Other,150,150,0.00,0.00
4,0.00,1 (0.2),Work,50,22,0.00,0.00
...,...,...,...,...,...,...,...
65,1.00,2 (0.3),Other,50,50,0.00,0.00
66,1.00,2 (0.3),Other,150,22,0.00,0.00
67,1.00,2 (0.3),Other,150,150,0.00,0.00
68,1.00,2 (0.3),Work,50,22,0.00,0.00


## 3. Compare the simulation results with existing charging infrastructure in Sweden

In [5]:
df_inf_comp = pd.read_csv(f'results/sensitivity/{scenario}_inf_comp.csv')
df_inf_comp.head()

,deso,Charging_type,power_fast,sim,gt
0,0114C1050,1 (0.2),50,2.86,0.00
1,0114C1050,1 (0.2),150,2.86,0.00
2,0114C1050,1 (0.3),50,2.86,0.00
3,0114C1050,1 (0.3),150,2.86,0.00
4,0114C1050,2 (0.2),50,2.86,0.00


1) Number of zones need more infrastructure, and
2) Maximum number of additional charging points needed.

In [6]:
import numpy as np
def stats_inf_comp(data):
    zone_share = len(data.loc[data['sim'] > data['gt'], :]) / len(data) * 100
    addi_num_max = max(data['sim'] - data['gt'])
    addi_num_median = np.median(data['sim'] - data['gt'])
    return pd.Series({'zone_share': zone_share, 'addi_num_max': addi_num_max, 'addi_num_median': addi_num_median})
df_inf_comp.groupby(['Charging_type', 'power_fast']).apply(stats_inf_comp).to_clipboard()
df_inf_comp.groupby(['Charging_type', 'power_fast']).apply(stats_inf_comp)

zone_share  addi_num_max  addi_num_median
Charging_type power_fast                                           
1 (0.2)       50               93.56         65.00             5.71
              150              93.36         59.29             5.71
1 (0.3)       50               96.86        155.71             8.57
              150              97.18        150.57             8.57
2 (0.2)       50               97.93        198.86            11.43
              150              98.02        184.86            11.43
2 (0.3)       50               98.27        193.43            11.43
              150              98.51        216.29            11.43
3 (0.9)       50               99.35        378.86            20.00
              150              99.35        376.00            20.00